In [ ]:
metrics_file = "metrics/application_1601392010735_0030"

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None



In [ ]:
import pyspark
import pyspark.sql.functions as F
import json

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
metrics = spark.read.json(metrics_file)

In [ ]:
metrics.select("App ID", "App Name").dropna().collect()

In [ ]:
def collect_and_dictify(df):
    return [json.loads(row[0]) for row in df.selectExpr("to_json(*)").collect()]

def executor_info(df):
    info = df.select("Executor Info").dropna()
    return collect_and_dictify(info)

def plan_dicts(df):
    return collect_and_dictify(df.select("sparkPlanInfo").dropna())

In [ ]:
from collections import namedtuple

MetricNode = namedtuple("MetricNode", "plan_node accumulatorId metricType name")
PlanInfoNode = namedtuple("PlanInfoNode", "plan_node parent nodeName simpleString")

def nextid():
    i = 0
    while True:
        yield i
        i = i + 1
    
node_ctr = nextid()

def plan_dicts(df):
    return collect_and_dictify(df.select("sparkPlanInfo").dropna())

def flatplan(dicts, parent=-1, plan_nodes=None, metric_nodes=None):
    if plan_nodes is None:
        plan_nodes = list()
        
    if metric_nodes is None:
        metric_nodes = list()
    
    for pd in dicts:
        pid = next(node_ctr)
        for m in pd['metrics']:
            metric_nodes.append(MetricNode(pid, m['accumulatorId'], m['metricType'], m['name']))
        
        plan_nodes.append(PlanInfoNode(pid, parent, pd['nodeName'], pd['simpleString']))
        
        flatplan(pd['children'], pid, plan_nodes, metric_nodes)
    
    return(plan_nodes, metric_nodes)

def plan_dfs(df):
    pn, mn = flatplan(plan_dicts(metrics))
    
    pndf = spark.createDataFrame(data=pn)
    mndf = spark.createDataFrame(data=mn)
    
    return (pndf, mndf)

In [ ]:
plan_nodes, accumulable_nodes = plan_dfs(metrics)

In [ ]:
pn, mn = flatplan(plan_dicts(metrics))

In [ ]:
metrics.select("Event").distinct().toPandas()

In [ ]:
def stageInfo(df):
    
    return collect_and_dictify(df.select("Stage Info").dropna())

In [ ]:
stageInfo(metrics)

In [ ]:
def accumulables(df, mcol='Task Info', idcol='Task ID'):
    acc_cols = [F.col('Accumulable.%s' % s).alias('Metric %s' % s) for s in ['ID', 'Name', 'Value']]
    obs = df.select(mcol).dropna().select('%s.*' % mcol)
    cols = [F.col(elt) for elt in sorted(set(obs.columns) - set([idcol, 'Accumulables']))]

    return obs.select(
        idcol, 
        F.explode('Accumulables').alias('Accumulable'), 
        *cols
    ).select(
        idcol, 
        *(cols + acc_cols)
    ).withColumnRenamed("Metric ID", "accumulatorId")

def tidy_metrics(df, mcol='Task Info', idcol='Task ID', interesting_metrics=None):
    acc_cols = [F.col('Accumulable.%s' % s).alias('Metric %s' % s) for s in ['ID', 'Name', 'Value']]
    obs = df.select(mcol).dropna().select('%s.*' % mcol)
    cols = [F.col(elt) for elt in sorted(set(obs.columns) - set([idcol, 'Accumulables']))]
    
    if interesting_metrics is None:
        interesting_metrics = F.col('Metric Name').isin(
            'internal.metrics.resultSerializationTime',
            'write time',
            'shuffle write time',
            'join time',
            'GPU time',
            'GPU decode time',
            'fetch wait time',
            'internal.metrics.executorCpuTime',
            'internal.metrics.executorDeserializeTime',
            'internal.metrics.jvmGCTime',
            'internal.metrics.jvmGCTime'
        )
    elif isinstance(interesting_metrics, list):
        interesting_metrics = F.col('Metric Name').isin(*interesting_metrics)
    elif isinstance(interesting_metrics, str):
        interesting_metrics = F.col('Metric Name').isin(interesting_metrics)
    
    return accumulables(df, mcol, idcol).where(interesting_metrics)

def tidy_tasks(df):
    return tidy_metrics(df, 'Task Info', 'Task ID', interesting_metrics=F.lit(True))

def tidy_stages(df):
    return tidy_metrics(df, 'Stage Info', 'Stage ID', interesting_metrics=F.lit(True))


In [ ]:
acc = accumulables(metrics, 'Task Info', 'Task ID')
[r[0] for r in acc.select('Metric Name').distinct().orderBy('Metric Name').collect()]


In [ ]:
MetricMeta = namedtuple('MetricMeta', 'MetricName kind unit')

metric_metas = [
    MetricMeta('GPU decode time', 'time', 'ms'),
    MetricMeta('GPU time', 'time', 'ms'),
    MetricMeta('avg hash probe bucket list iters', 'count', 'iterations'),
    MetricMeta('buffer time', 'time', 'ms'),
    MetricMeta('build side size', 'size', 'bytes'),
    MetricMeta('build time', 'time', 'ms'),
    MetricMeta('collect batch time', 'time', 'ms'),
    MetricMeta('concat batch time', 'time', 'ms'),
    MetricMeta('data size', 'size', 'bytes'),
    MetricMeta('duration', 'time', 'ms'),
    MetricMeta('fetch wait time', 'time', 'ms'),
    MetricMeta('internal.metrics.diskBytesSpilled', 'size', 'bytes'),
    MetricMeta('internal.metrics.executorCpuTime', 'time', 'ns'),
    MetricMeta('internal.metrics.executorDeserializeCpuTime', 'time', 'ns'),
    MetricMeta('internal.metrics.executorDeserializeTime', 'time', 'ms'),
    MetricMeta('internal.metrics.executorRunTime', 'time', 'ms'),
    MetricMeta('internal.metrics.input.bytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.input.recordsRead', 'count', 'records'),
    MetricMeta('internal.metrics.jvmGCTime', 'time', 'ms'),
    MetricMeta('internal.metrics.memoryBytesSpilled', 'size', 'bytes'),
    MetricMeta('internal.metrics.output.bytesWritten', 'size', 'bytes'),
    MetricMeta('internal.metrics.output.recordsWritten', 'count', 'records'),
    MetricMeta('internal.metrics.peakExecutionMemory', 'size', 'bytes'),
    MetricMeta('internal.metrics.resultSerializationTime', 'time', 'ms'),
    MetricMeta('internal.metrics.resultSize', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.fetchWaitTime', 'time', 'ms'),
    MetricMeta('internal.metrics.shuffle.read.localBlocksFetched', 'count', 'blocks'),
    MetricMeta('internal.metrics.shuffle.read.localBytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.recordsRead', 'count', 'records'),
    MetricMeta('internal.metrics.shuffle.read.remoteBlocksFetched', 'count', 'blocks'),
    MetricMeta('internal.metrics.shuffle.read.remoteBytesRead', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.read.remoteBytesReadToDisk', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.write.bytesWritten', 'size', 'bytes'),
    MetricMeta('internal.metrics.shuffle.write.recordsWritten', 'count', 'records'),
    MetricMeta('internal.metrics.shuffle.write.writeTime', 'time', 'ms'),
    MetricMeta('join output rows', 'count', 'rows'),
    MetricMeta('join time', 'time', 'ms'),
    MetricMeta('local blocks read', 'count', 'blocks'),
    MetricMeta('local bytes read', 'size', 'bytes'),
    MetricMeta('number of input columnar batches', 'count', 'batches'),
    MetricMeta('number of input rows', 'count', 'rows'),
    MetricMeta('number of output columnar batches', 'count', 'batches'),
    MetricMeta('number of output rows', 'count', 'rows'),
    MetricMeta('peak device memory', 'size', 'bytes'),
    MetricMeta('peak memory', 'size', 'bytes'),
    MetricMeta('records read', 'count', 'records'),
    MetricMeta('remote blocks read', 'count', 'blocks'),
    MetricMeta('remote bytes read', 'size', 'bytes'),
    MetricMeta('scan time', 'time', 'ms'),
    MetricMeta('shuffle bytes written', 'size', 'bytes'),
    MetricMeta('shuffle records written', 'count', 'records'),
    MetricMeta('shuffle write time', 'time', 'ms'),
    MetricMeta('spill size', 'size', 'bytes'),
    MetricMeta('sort time', 'time', 'ms'),
    MetricMeta('time in aggregation build', 'time', 'ms'),
    MetricMeta('time in batch concat', 'time', 'ms'),
    MetricMeta('time in compute agg', 'time', 'ms'),
    MetricMeta('total time', 'time', 'ns'),
    MetricMeta('write time', 'time', 'ms')
]

In [ ]:
metric_meta = spark.createDataFrame(data=metric_metas)

In [ ]:
task_metrics = tidy_tasks(metrics)

In [ ]:
task_metrics.select("accumulatorId").distinct().count()

In [ ]:
accumulable_nodes.select("accumulatorId").distinct().count()

In [ ]:
task_metrics.select("accumulatorId").union(accumulable_nodes.select("accumulatorId")).distinct().count()



In [ ]:
stage_metrics = tidy_stages(metrics)
stage_metrics_outer = stage_metrics.join(accumulable_nodes, "accumulatorId", how="outer")
stage_metrics_outer.where(F.col("name").isNull()).toPandas()


In [ ]:
stage_metrics.printSchema()

In [ ]:
task_metrics_outer = task_metrics.join(accumulable_nodes, "accumulatorId", how="outer")
task_metrics_outer.where(F.col("name").isNull()).toPandas()

In [ ]:
task_metrics_outer.where(F.col("name").isNull()).select("Metric Name").distinct().toPandas()

In [ ]:
tasks_to_plans = task_metrics.join(accumulable_nodes, "accumulatorId").join(plan_nodes, "plan_node")


In [ ]:
tasks_to_plans.count()

In [ ]:
task_metrics.select("Metric Name").distinct().collect()

In [ ]:
task_metrics = tidy_tasks(metrics).toPandas()

In [ ]:
task_metrics

In [ ]:
import altair as alt

In [ ]:
alt.Chart(task_metrics).mark_bar().encode(
    x='Task ID:N',
    y=alt.Y('sum(Metric Value):Q'),
    color='Metric Name:N',
    tooltip=['Metric Name', 'Metric Value', 'Task ID']
).interactive()

In [ ]:
alt.Chart(task_metrics).mark_bar().encode(
    x='Task ID:N',
    y=alt.Y('sum(Metric Value):Q', stack="normalize"),
    color='Metric Name:N',
    tooltip=['Metric Name', 'Metric Value', 'Task ID']
).interactive()

In [ ]:
stage_metrics = tidy_stages(metrics).toPandas()
alt.Chart(stage_metrics).mark_bar().encode(
    x='Stage ID:N',
    y='Metric Value:Q',
    color='Metric Name:N',
    tooltip=['Details', 'Metric Name', 'Metric Value', 'Stage ID']
).interactive()

In [ ]:
stage_metrics = tidy_stages(metrics).toPandas()
alt.Chart(stage_metrics).mark_bar().encode(
    x='Stage ID:N',
    y=alt.Y('sum(Metric Value):Q', stack="normalize"),
    color='Metric Name:N',
    tooltip=['Details', 'Metric Name', 'Metric Value', 'Stage ID']
).interactive()

In [ ]:
stage_metrics

In [ ]:
metrics.select("System Properties").dropna().collect()

In [ ]:
def melt(df, id_vars = None, value_vars=None, var_name='variable', value_name='value'):
    if id_vars is None:
        id_vars = []
    
    if value_vars is None:
        value_vars = [c for c in df.columns if c not in id_vars]
    
    return df.withColumn(
        "value_tuple",
        F.explode(
            F.array(
                *[
                    F.struct(
                        F.lit(vv).alias(var_name), 
                        F.col("`%s`" % vv).alias(value_name)
                    ) 
                    for vv in value_vars
                ]
            )
        )
    ).select(*(id_vars + [F.col("value_tuple")[cn].alias(cn) for cn in [var_name, value_name]]))

In [ ]:
melt(metrics.select("Properties").dropna().select("Properties.*")).dropna().collect()

In [ ]:
pdf = metrics.where(F.col("Properties").isNotNull()).toPandas()

In [ ]:
metrics.select("Properties").dropna().select("Properties.*").select("`spark.app.id`").distinct().collect()

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pdf

In [ ]:
psi = metrics.where(F.col("Properties").isNotNull()).select("Stage Info.*").toPandas()

In [ ]:
psi